# MODIS EVI over Continental US: 0.5x0.5 degree grid, 4x5 monthly average support

For use a covariate in mapping SIF predictions back to the data scale.

In [1]:
import sys
sys.path.insert(0, "../src")

from functools import reduce

import numpy as np
import pandas as pd 
import xarray as xr

from data_utils import augment_dataset_pred

In [2]:
ds_evi = xr.open_dataset("../data/exp_pro/MODIS_EVI_conus.nc", engine="h5netcdf")
ds_evi

<xarray.Dataset>
Dimensions:  (time: 72, lon: 1400, lat: 880)
Coordinates:
  * lat      (lat) float64 61.97 61.92 61.88 61.83 ... 18.17 18.12 18.08 18.02
  * lon      (lon) float64 -130.0 -129.9 -129.9 -129.8 ... -60.12 -60.07 -60.02
  * time     (time) datetime64[ns] 2015-01-01 2015-02-01 ... 2020-12-01
Data variables:
    evi      (time, lon, lat) float32 ...

In [3]:
# Computed the gridded, aggregated dataset
# config = GridConfig((-125, -65, 22, 58), lon_res=0.5, lat_res=0.5)
# df = prep_gridded_df(ds_evi, config)
# df

df = augment_dataset_pred(ds_evi)
df

/Users/joshjacobson/Projects/sif-data-fusion/scripts/../src/data_utils.py:132: UserWarning: Neither offset is zero.
  warnings.warn("Neither offset is zero.")


,lon,lat,time,evi
0,-127.0,44.5,2015-01-01,33911668.0
1,-127.0,44.5,2015-02-01,35113332.0
2,-127.0,44.5,2015-03-01,32191428.0
3,-127.0,44.5,2015-04-01,38386668.0
4,-127.0,44.5,2015-05-01,40610000.0
...,...,...,...,...
6691,-67.5,56.0,2020-08-01,32095260.0
6692,-67.5,56.0,2020-09-01,29848782.0
6693,-67.5,56.0,2020-10-01,14236548.0
6694,-67.5,56.0,2020-11-01,6237495.0


In [4]:
ds = df.set_index(["lon", "lat", "time"]).to_xarray()
ds

<xarray.Dataset>
Dimensions:  (lon: 129, lat: 79, time: 72)
Coordinates:
  * lon      (lon) float64 -127.0 -126.5 -126.0 -125.5 ... -64.0 -63.5 -63.0
  * lat      (lat) float64 20.5 21.0 21.5 22.0 22.5 ... 57.5 58.0 58.5 59.0 59.5
  * time     (time) datetime64[ns] 2015-01-01 2015-02-01 ... 2020-12-01
Data variables:
    evi      (lon, lat, time) float32 nan nan nan ... -4.751e+06 -6.905e+06

In [5]:
ds.to_netcdf("../data/exp_pro/MODIS_05deg_monthly_conus_new.nc", format="NETCDF4")
ds.close()